# Kalman filter
In this live script, we iterate the equations of the Kalman filter, for a 
given example. We do so for a large number of iterations, so that the Kalman 
filter gains converge to a stationary solution, and compare the solution to 
the one obtained with the function `kalman`.

Let us start by recalling the Kalman filter. Consider a linear process

$$x_{k+1}=A x_{k}+B u_{k}+w_{k}$$

$$y_{k} =C x_{k}+n_{k} $$

where

$x_{0}\sim \mathcal{N}\left(\bar{x}_{0},\bar{\Phi}_{0}\right)$,$w_{k}\sim \mathcal{N}(0, W)$,$n_{k} \sim \mathcal{N}(0, V)$.
Consider also the information set

$I_{k}  {=\left(y_{0}, y_{1}, \ldots, y_{k}, u_{0}, u_{1}, \ldots, u_{k-1}\right)} 
$, $ {k \geq 0}$.

Then 

$$P_{x_{k}} | I_{k} \sim \mathcal{N}\left(\hat{x}_{k | k}, \Phi_{k | k}\right)$$

where

$$\hat{x}_{k | k} =\hat{x}_{k | k-1}+L_{k}\left(y_{k}-C \hat{x}_{k | k-1}\right)$$

$$\hat{x}_{k +1| k} =A\hat{x}_{k | k}+Bu_k$$

$$\Phi_{k | k} =\Phi_{k | k-1}-\Phi_{k | k-1} C^{\top}\left(C \Phi_{k | k-1} C^{\top}+V\right)^{-1} C \Phi_{k | k-1}$$

$$ \Phi_{k+1 | k} =A \Phi_{k | k} A^{\top}+W $$

$$L_{k}=\Phi_{k | k-1} C^{\top}\left(C \Phi_{k | k-1} C^{\top}+V\right)^{-1}$$

with initial condition $\hat{x}_{0|-1}=\bar{x}_0$ , $\Phi_{0|-1}=\bar{\Phi}_0$.

As $k\rightarrow \infty$, $\Phi_{k+1|k}\rightarrow \bar{\Phi}$, $L_k\rightarrow\bar{L}$, 
where $\bar{\Phi}$ satisfies

$$\bar{\Phi}=A \bar{\Phi} A^{\top} -A \bar{\Phi} C^{\top} \left(C \bar{\Phi} C^{\top}+V\right)^{-1} C \bar{\Phi} A^{\top}+W$$

and $\bar{L}$ is given by

$${\bar{L}=\bar{\Phi} C^{\top}\left(C \bar{\Phi} C^{\top}+V\right)^{-1}}$$

The following example iterates these Kalman filter equations for the model

$$\dot{x}(t)=\underbrace{\left[\begin{array}{cc}{0} & {1} \\ 
{-w_{n}^{2}} & {0}\end{array}\right]}_{A_{c}} x(t)$$

which, after discretization with sampling period $\tau = 0.1$, leads to

$$x_{k+1}=\underbrace{\left[\begin{array}{cc}{0.8090} & {0.0935} \\ {-3.6932} & {0.8090}\end{array}\right]}_{A=e^{A_{c} \tau}} x_{k}+\underbrace{\left[\begin{array}{c}{0} \\ {\sigma_{w}}\end{array}\right] d_{k}}_{w_{k}}$$

where additive disturbances have been considered in discrete time and $d_k$ 
is a Gaussian disturbance for every $k$. The considered output  is

$$y_{k}=\underbrace{\left[\begin{array}{ll}{1} & {0}\end{array}\right]}_{C} 
x_{k}+\underbrace{\sigma_{n} v_{k}}_{n_{k}}$$

Besides comparing the stationary solution obtained by iterating the Kalman 
filter equations and the solution obtained with Kalman.m, this script also plots 
some initial estimates $\hat{x}_{k|k-1}$ and $\hat{x}_{k|k}$ and the elipsoids

$$\begin{array}{l}{\left\{\frac{1}{4} \sqrt{\Phi_{k | k-1}} x |\|x\|=1\right\}} \\
{\left\{\frac{1}{4} \sqrt{\Phi_{k | k}} x |\|x\|=1\right\}}\end{array}$$

for $k=0$.


In [ ]:
import numpy as np
from scipy import signal
import scipy.linalg
import matplotlib.pyplot as plt
from course_functions import kalman

In [ ]:
def kalmangains(A,C,Theta0,W,V,kend):
    A = np.atleast_2d(A)
    C = np.atleast_2d(C)    
    L = [np.zeros((2,1)) for id in range(kend)]
    Theta = [np.eye(2) for id in range(kend)]
    Theta_1 = [np.eye(2) for id in range(kend+1)]
    Theta_1[0] = Theta0
    for k in range(kend):
        L[k] = Theta_1[k]@C.T@np.linalg.inv(C@Theta_1[k]@C.T+V)
        Theta[k] = Theta_1[k] - Theta_1[k]@C.T@np.linalg.inv( C@Theta_1[k]@C.T +V )@C@Theta_1[k]
        Theta_1[k+1] = A@Theta[k]@A.T + W
    
    return L, Theta, Theta_1

In [ ]:
np.random.seed(1)

# system definition
ts = 100
ks = (2*np.pi)**2
cf = 0
Ac = np.array([[0, 1],[-ks, -cf]])
n = 2
Bc = np.array([[0], [1]])
Cc = np.array([1, 0])
Dc = 0

#discretization
tau = 0.1
Ad, Bd, Cd, Dd = signal.cont2discrete((Ac, Bc, Cc, Dc), tau)[:4]

#Kalman filter - obtain the time-varying gains
sigmaw = 0.5
sigman = 0.1
Theta0 = np.eye(2)
Theta_1 = [np.eye(2) for idx in range(1002)]
Theta_1[0] = Theta0
W = np.diag(np.array([0, sigmaw**2]))
V = sigman**2
kend = int(ts/tau+1)
L, Theta, Theta_1 = kalmangains(Ad,Cd,Theta0,W,V,kend)

In [ ]:
#simulation
x0 = np.array([[1],[0]])
x = np.zeros((2,1002))
x[:,[0]] = x0
xhat_1 = np.zeros((2,1002))
xhat_1[:,[0]] = np.array([[1.3], [0]])
y = np.zeros((1,1001))
xhat = np.zeros((2,1001))
for k in range(kend):
    y[:,[k]] = Cd@x[:,[k]]+sigman*np.random.randn()
    xhat[:,[k]] = xhat_1[:,[k]] + L[k]@(y[:,[k]]-Cd@xhat_1[:,[k]])
    xhat_1[:,[k+1]] = Ad@xhat[:,[k]]
    x[:,[k+1]] = Ad@x[:,[k]] + np.array([[0],[sigmaw*np.random.randn()]])

KF = 11
f = plt.figure()
ax = plt.gca()
ax.plot(x[0,0:KF-1],x[1,0:KF-1],'o')
ax.plot(xhat[0,0:KF-1],xhat[1,0:KF-1],'rx')
ax.plot(xhat_1[0,0:KF-1],xhat_1[1,0:KF-1],'g+')
ax.axis('equal')
ax.grid(True)

In [ ]:
#plot ellipsoid 1
f = plt.figure()
ax = plt.gca()
c = xhat_1[:,[0]]
MM = scipy.linalg.sqrtm(Theta_1[0])/4
thetas = np.arange(0,2*np.pi,0.01)
ps = np.vstack((np.cos(thetas),np.sin(thetas)))
for ind in range(len(thetas)):
    pi_ = c +MM@ps[:,[ind]]
    ax.plot(pi_[0],pi_[1],'g.')
    
ax.axis('equal')
ax.grid(True)

#plot ellipsoid 2
c = xhat[:,[0]]
MM = scipy.linalg.sqrtm(Theta[0])/4
thetas = np.arange(0,2*np.pi,0.01)
ps = np.vstack((np.cos(thetas),np.sin(thetas)))
for ind in range(len(thetas)):
    pi_ = c + MM@ps[:,[ind]]
    ax.plot(pi_[0],pi_[1],'r.')

In [ ]:
# infinite horizon
G = np.eye(n)
a,Thetainf,c,Linf = kalman(Ad, G, Cd, W, V, continuous=False)  

Stationary solution Ric. eq. and stationary filter gains

In [ ]:
Thetainf

In [ ]:
Linf

Theta_{k|k-1}, L_k for large k

In [ ]:
Theta_1[-1]

In [ ]:
L[-1]